In [19]:
import cv2 
import uuid
import re
import os
import time

# 1. Define Images Label

In [20]:
COLLECTED_IMG_PATH = r'.\Tensorflow\workspace\images\collectedimages'

labels = os.listdir(COLLECTED_IMG_PATH)
labels

['botolkaca',
 'botolplastik',
 'kaleng',
 'kardus',
 'karet',
 'kertas',
 'plastik',
 'sedotan']

# 2. Setup Folders 

In [21]:
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

if not os.path.exists(IMAGES_PATH):
    !mkdir {IMAGES_PATH}
    
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        os.mkdir(path)

# 3. Image Labelling

In [23]:
# !pip install --upgrade pyqt5 lxml

In [24]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [26]:
if os.name == 'posix':
    !make qt5py3
if os.name =='nt':
    !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
# Nanti muncul python gui, pakai python gui itu buat extract fitur
!cd {LABELIMG_PATH} && python labelImg.py

# 4. Copy them into a Training and Testing Partition

First delete the contents of the train and test directories!!

In [30]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')

if not os.path.exists(TRAIN_PATH):
    os.mkdir(TRAIN_PATH)

if not os.path.exists(TEST_PATH):
    os.mkdir(TEST_PATH)

# 5. Splitting Data to Train and Test

In [31]:
# Shuffle and copy collected image to train and test dir
import random
import shutil

TRAIN_TO_TOTAL_RATIO = 0.8
RANDOM_SEED = 101

random.seed(RANDOM_SEED)

#Creating list for train and test
total = 0
for file_folder_name in labels:
    xml_list = []
    file_for_test = []
    file_dir = os.path.join(IMAGES_PATH, file_folder_name)
    xml_list = [file for file in os.listdir(file_dir) if file.endswith('xml')]
    train_num = int((TRAIN_TO_TOTAL_RATIO*len(xml_list))//1)
    file_for_training = list(random.sample(xml_list, train_num))
    file_for_test = [file for file in xml_list if file not in file_for_training]
    
    # Verbose
    name = file_folder_name if len(file_folder_name) > 7 else file_folder_name + '\t'
    print(f"{name}\t | Train : {len(file_for_training)}\tTest : {len(file_for_test)}")
    total += (len(file_for_training) + len(file_for_test))
    
    #copy file to training
    for file in file_for_training:
        file_xml = os.path.join(file_dir, file)
        file_jpg = os.path.join(file_dir, file[:-3]) + 'jpg'
        jpg_target = os.path.join(TRAIN_PATH, file[:-3]) + 'jpg'
        xml_target = os.path.join(TRAIN_PATH, file)
        shutil.copyfile(file_xml, xml_target)
        shutil.copyfile(file_jpg, jpg_target)
    
    #copy file to test
    for file in file_for_test:
        file_xml = os.path.join(file_dir, file)
        file_jpg = os.path.join(file_dir, file[:-3]) + 'jpg'
        jpg_target = os.path.join(TEST_PATH, file[:-3]) + 'jpg'
        xml_target = os.path.join(TEST_PATH, file)
        shutil.copyfile(file_xml, xml_target)
        shutil.copyfile(file_jpg, jpg_target)
    
print(f"Total data : {total}")

botolkaca	 | Train : 398	Test : 100
botolplastik	 | Train : 409	Test : 103
kaleng		 | Train : 573	Test : 144
kardus		 | Train : 454	Test : 114
karet		 | Train : 426	Test : 107
kertas		 | Train : 334	Test : 84
plastik		 | Train : 363	Test : 91
sedotan		 | Train : 464	Test : 116
Total data : 4280


Cek ukuran test dan train

In [33]:
print(f"Train data : {len(os.listdir(TRAIN_PATH))} data")
print(f"Test data : {len(os.listdir(TEST_PATH))} data")
print(f"Total data : {(len(os.listdir(TRAIN_PATH)) + len(os.listdir(TEST_PATH))) / 2}")

Train data : 6842 data
Test data : 1718 data
Total data : 4280.0
